In [1]:
from tkinter import *
from tkinter.ttk import *
import tkinter.font as font
from tkinter import filedialog
import os
from PIL import ImageTk, Image
import tensorflow as tf
import cv2
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
from keras.models import load_model
import PIL.Image
import tkinter as tk

In [2]:
import warnings
warnings.filterwarnings(action='ignore')
import re
import pymongo
import pandas as pd
from konlpy.tag import Okt
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import datasets, layers, models, optimizers

In [25]:
window = Tk()

window.title('런웨이와 챗봇상담')
window.geometry("850x900+0+100")

def pressed():
    label.configure(text="버튼을 누름")

def id_q():
    chat_bot= "runway: 아이디를 입력하세요"
    return chat_bot

def category_list():
    chat_bot= "runway: 확인 가능한 목록은 다음과 같습니다 \n 1(총 구매 목록), 2(구매 상품), 3(총 구매 가격), 4(결제 건수), 5(특정 시간에 따른 구매 목록)"
    return chat_bot

def id_explain():
    Output.insert(END,id_q() + '\n')

def list_explain(): # 설명 보기
    Output.insert(END,category_list() + '\n')

def date_q():
    chat_bot= "runway: 조회하려는 날짜를 선택하세요 (YYYY-mm-dd 형식으로 입력해주세요)"
    return chat_bot
## -------------------- 변수 선언
def id_input():
    input_id = "user : " + input_text2.get()
#     Output.insert(END,input_id + '\n')
    return input_id

def check_input():
    check = "user : " + input_text3.get()
#     Output.insert(END,check + '\n')
    return check

def search_date_input():
    input_date = "user : " + input_text4.get()
    Output.insert(END,input_date + '\n')
    return input_date
## -------------------- 변수 선언

def result():
    check_num = int(input_text3.get())
    if check_num == 1:
        result = check_one()
        return result
    elif check_num == 2:
        result = check_two()
        return result
    elif check_num == 3:
        result = check_three()
        return result
    elif check_num == 4:
        result = check_four()
        return result
    elif check_num == 5:
        result = check_five()
        return result
    else:
        result = error()
        return result
    
def check_one():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbot
    purchaselist = chatbot.PurchaseList
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    search_list = list[list['user_id']==str(input_text2.get())]
    chat_bot= "runway: 고객님의 구매목록은 다음과 같습니다"
    search_list = pd.DataFrame(search_list, 
                                     columns=['purchase_product', 'payment_method', 'payment_total', 'payment_date'])
    Output.insert(END,chat_bot + '\n')
    Output.insert(END,search_list + '\n')
    return chat_bot, search_list

def check_two():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbot
    purchaselist = chatbot.PurchaseList
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    search_product = list[list['user_id']==str(input_text2.get())]['purchase_product']
    chat_bot= "runway: 고객님이 현재까지 구매하신 상품은 다음과 같습니다."
    Output.insert(END,chat_bot + '\n')
    purchase_list = ','.join(search_product.to_list())
    Output.insert(END, purchase_list + '\n')
    return chat_bot, purchase_list

def check_three():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbot
    purchaselist = chatbot.PurchaseList
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    search_tprice = list[list['user_id']==str(input_text2.get())]['payment_total'].astype(int).sum()
    chat_bot= "runway: 고객님의 총 구매 가격은 " + str(search_tprice) + "입니다"
    Output.insert(END,chat_bot + '\n')
    Output.insert(END,search_tprice + '\n')
    return chat_bot, search_tprice
                  
def check_four():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbot
    purchaselist = chatbot.PurchaseList
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    list['payment_total'] = list['payment_total'].astype(int)
    total_count = list[list['user_id']==str(input_text2.get())][['payment_method', 'payment_total']]
    search_payment_method_credit = total_count[total_count['payment_method'] == '신용카드']['payment_method'].unique()[0]
    search_payment_total_credit = total_count[total_count['payment_method'] == '신용카드']['payment_total'].sum()
    search_payment_method_noaccount = total_count[total_count['payment_method'] == '무통장입금']['payment_method'].unique()[0]
    search_payment_total_noaccount = total_count[total_count['payment_method'] == '무통장입금']['payment_total'].sum()
    chat_bot= "runway: 조회하신 id에 대한 결제 건수는, " + search_payment_method_credit + ": " + str(search_payment_total_credit) + "원, " + search_payment_method_noaccount + ": " + str(search_payment_total_noaccount) + "원입니다."
    Output.insert(END,chat_bot + '\n')
    return chat_bot

def check_five():
    import pymongo
    import pandas as pd
    conn = pymongo.MongoClient()
    chatbot = conn.chatbot
    purchaselist = chatbot.PurchaseList
    list = pd.DataFrame()
    docs = purchaselist.find({})
    for doc in docs:
        df = pd.DataFrame(docs, columns={'user_id', 'purchase_product', 'payment_method', 'payment_total', 'payment_date'})
    list = list.append(df)
    Input = date_q()
    search_date_list = list[(list['user_id']==str(input_text2.get())) & (list['payment_date'].str.contains(str(input_text4.get())))]
    chat_bot= "runway: 조회하신 날짜에 대한 구매목록은 다음과 같습니다."
    Output.insert(END,chat_bot + '\n')
    Output.insert(END,search_date_list.to_string(index=False))
    return chat_bot

def error():
    chat_bot= "runway: 챗봇을 종료합니다. 이용해주셔서 감사합니다"
    Output.insert(END,chat_bot + '\n')
    return chat_bot
    
## -----------------런웨이 말하는 부분
path="C:/Users/hi/Desktop/DeepLearning/image/runway.png"
if path == '':exit()
image=Image.open(path)
height=50
width=70
canvas=Canvas(window, height=500, width=500)
image=image.resize((height,width),PIL.Image.ANTIALIAS)
photo=ImageTk.PhotoImage(image)
label2=Label(window, image=photo) #라벨 생성, 라벨에는 앞서 선언한 이미지가 들어감.
label2.pack()  #runway image

## -----------------채팅 내용칸
frame2 = Frame(window) 
frame2.pack(side=tk.TOP)
Output = Text(frame2, height = 36,width =100,bg = "light cyan")
Output.configure(font=("Courier", 12, "italic"))
Output.grid(column=0, row=0)

## -----------------채팅 입력칸
frame1 = Frame(window)
frame1.pack(side=tk.BOTTOM)
button = Button(frame1, text="안내받기", command=id_explain)
button.grid(column=0, row=2)
button = Button(frame1, text="설명 보기", command=list_explain)
button.grid(column=1, row=2)
button = Button(frame1, text="결과 보기", command=result)
button.grid(column=1, row=3)
input_text2 = Entry(frame1, width=60)
input_text2.grid(column=0, row=4)
button = Button(frame1, text="id 입력", command=id_input)
button.grid(column=1, row=4)
input_text3 = Entry(frame1, width=60)
input_text3.grid(column=0, row=5)
button = Button(frame1, text="선택지 입력", command=check_input)
button.grid(column=1, row=5)
input_text4 = Entry(frame1, width=60)
input_text4.grid(column=0, row=6)
button = Button(frame1, text="날짜 입력", command=search_date_input)
button.grid(column=1, row=6)

window.mainloop()